In [2]:
import numpy as np
import pandas as pd

In [5]:
train_df = pd.read_parquet('/kaggle/input/train-parquet')
labels = pd.read_parquet('/kaggle/input/how-to-get-32gb-ram/train_labels.parquet')
all_labels = pd.read_csv('/kaggle/input/all-labels-csv/session_df.csv')

**Hypothesis Testing Example for music and correctness of question 1**

In [6]:
music = train_df[['session_id','music']].drop_duplicates(ignore_index = True)

In [7]:
q1 = labels[labels['session_id'].str[-2:]=='q1']
q1['session_id'] = q1['session_id'].str[:-3]
q1['session_id'] = q1['session_id'].astype(int)

/tmp/ipykernel_48/2566661744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['session_id'] = q1['session_id'].str[:-3]
/tmp/ipykernel_48/2566661744.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['session_id'] = q1['session_id'].astype(int)


In [8]:
merged_df = pd.merge(music, q1, on='session_id', how='left')
merged_df = merged_df[['music','correct']]

In [9]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(merged_df['music'], merged_df['correct'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-squared test p-value:", p)
alpha = 0.05 
if p < alpha:
    print("Reject the null hypothesis - there is an association between listening to music and correctness of answers.")
else:
    print("Fail to reject the null hypothesis - there is no evidence of an association between listening to music and correctness of answers.")

Chi-squared test p-value: 0.2433117270812743
Fail to reject the null hypothesis - there is no evidence of an association between listening to music and correctness of answers.


**Hypothesis Testing for the correct rate of all questions**


In [41]:
all_labels = pd.read_csv('/kaggle/input/all-labels-csv/session_df.csv')

In [42]:
all_labels['rate']=all_labels['question_vector'].apply(lambda x:x.count('1'))/18

In [43]:
m = all_labels['rate'].median()

In [48]:
performance = all_labels[['session_id','rate']]

In [49]:
performance['result']=performance['rate'].apply(lambda x: 1 if x>=m else 0)

/tmp/ipykernel_48/3682850510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance['result']=performance['rate'].apply(lambda x: 1 if x>=m else 0)


In [57]:
merged_df = pd.merge(music, performance, on='session_id', how='left')
merged_df = merged_df[['music','result']]

In [61]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(merged_df['music'], merged_df['result'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-squared test p-value:", p)
alpha = 0.05 
if p < alpha:
    print("Reject the null hypothesis - there is an association between listening to music and correctness rate.")
else:
    print("Fail to reject the null hypothesis - there is no evidence of an association between listening to music and correctness rate.")

Chi-squared test p-value: 0.07410530884815965
Fail to reject the null hypothesis - there is no evidence of an association between listening to music and correctness rate.


In [64]:
hq = train_df[['session_id','hq']].drop_duplicates(ignore_index = True)

In [67]:
merged_df = pd.merge(hq, performance, on='session_id', how='left')
merged_df = merged_df[['hq','result']]
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(merged_df['hq'], merged_df['result'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-squared test p-value:", p)
alpha = 0.05 
if p < alpha:
    print("Reject the null hypothesis - there is an association between high quality mode of game and correctness rate.")
else:
    print("Fail to reject the null hypothesis - there is no evidence of an association between high quality mode of game and correctness rate.")

Chi-squared test p-value: 0.00021158865394536424
Reject the null hypothesis - there is an association between high quality mode of game and correctness rate.


In [69]:
fullscreen = train_df[['session_id','fullscreen']].drop_duplicates(ignore_index = True)
merged_df = pd.merge(fullscreen, performance, on='session_id', how='left')
merged_df = merged_df[['fullscreen','result']]
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(merged_df['fullscreen'], merged_df['result'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-squared test p-value:", p)
alpha = 0.05 
if p < alpha:
    print("Reject the null hypothesis - there is an association between fullscreen and correctness rate.")
else:
    print("Fail to reject the null hypothesis - there is no evidence of an association between fullscreen and correctness rate.")

Chi-squared test p-value: 0.5102269621889663
Fail to reject the null hypothesis - there is no evidence of an association between fullscreen and correctness rate.
